In [1]:
import snap 
import csv
import pandas as pd
import random
import networkx as nx
from node2vec import Node2Vec
#from deepwalk import DeepWalk
#from graphembedding.sdne import SDNE
#from gem.embedding.hope import HOPE

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import ParameterGrid
from scipy.sparse import coo_matrix


%run Fairness.ipynb
%run Embeddings_Classification.ipynb


Reading the path of the Dataset as a panda Dataframe

In [2]:
employee_path = 'Datasets/Employee/Employee.csv'


In [3]:
df = pd.read_csv(employee_path)
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [4]:
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [5]:
row, column = df.shape
print(row)
print(column)

4653
9


change the datatype of the attributues to integers (numbers = categories)

In [6]:
#change Gender male -> 0; female-> 1
for i in range(0, len(df['Gender'])):
    #print(df.at[i, 'GoodCustomer'])
    if df.at[i, 'Gender'] == 'Male':
        df.at[i, 'Gender'] = 0
    else:
        df.at[i, 'Gender'] = 1
#change Gender Bachelor -> 0; Master-> 1
for i in range(0, len(df['Education'])):
    if df.at[i, 'Education'] == 'Bachelors':
        df.at[i, 'Education'] = 0
    else:
        df.at[i, 'Education'] = 1
#change the Citys into categories        
for i in range(0, len(df['City'])):
    if df.at[i, 'City'] == 'Bangalore':
        df.at[i, 'City'] = 0
    elif df.at[i, 'City'] == 'Pune':
        df.at[i, 'City'] = 1
    else:
        df.at[i, 'City'] = 2
#change the Ever Benched attribute to integers (0 = no,1 = yes)
for i in range(0, len(df['EverBenched'])):
    if df.at[i, 'EverBenched'] == 'No':
        df.at[i, 'EverBenched'] = 0
    else:
        df.at[i, 'EverBenched'] = 1

In [7]:
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,0,2017,0,3,34,0,0,0,0
1,0,2013,1,1,28,1,0,3,1
2,0,2014,2,3,38,1,0,2,0
3,1,2016,0,3,27,0,0,5,1
4,1,2017,1,3,24,0,1,2,1


Count the females and males

In [8]:
count_0 = df['Gender'].value_counts().get(0, 0)
print(count_0)
count_1 = df['Gender'].value_counts().get(1, 0)
print(count_1)

2778
1875


In [9]:
df_target = df['LeaveOrNot']
df_gender = df['Gender']
df_education = df['Education']
df_joiningYear = df['JoiningYear']
df_City = df['City']
df_PaymentTier = df['PaymentTier']
df_Age = df['Age']
df_EverBenched = df['EverBenched']
df_ExperienceInCurrentDomain = df['ExperienceInCurrentDomain']

In [10]:
df_target.head()

0    0
1    1
2    0
3    1
4    1
Name: LeaveOrNot, dtype: int64

In [11]:
df_gender.head()

0    0
1    1
2    1
3    0
4    0
Name: Gender, dtype: object

In [12]:
# Number of nodes
num_nodes = row

# target-Density (z.B., 01%)
density = 0.001

# Compute the maximal number of possible edges
max_edges = (num_nodes * (num_nodes - 1)) / 2

# Compute the target number of edges
target_edges = int(density * max_edges)

In [13]:
# Create a graph and add random edges
G = nx.Graph()
nodes = list(range(num_nodes))
G.add_nodes_from(nodes)

In [14]:
print(G.number_of_nodes())


4653


Add Attributes to the nodes of the network

In [15]:
nx.set_node_attributes(G, df_gender, 'gender')
nx.set_node_attributes(G, df_education, 'Education')
nx.set_node_attributes(G, df_joiningYear, 'JoiningYear')
nx.set_node_attributes(G, df_City, 'City')
nx.set_node_attributes(G, df_PaymentTier, 'PaymentTier')
nx.set_node_attributes(G, df_Age, 'Age')
nx.set_node_attributes(G, df_EverBenched, 'EverBenched')
nx.set_node_attributes(G, df_ExperienceInCurrentDomain, 'ExperienceInCurrentDomain')
print(G.nodes[0])

{'gender': 0, 'Education': 0, 'JoiningYear': 2017, 'City': 0, 'PaymentTier': 3, 'Age': 34, 'EverBenched': 0, 'ExperienceInCurrentDomain': 0}


Add edges to the graph based on the target edges

In [16]:
while G.number_of_edges() < target_edges:
    node1, node2 = random.sample(nodes, 2)
    if not G.has_edge(node1, node2):
        G.add_edge(node1, node2)

In [17]:
# Number of generated edges
actual_edges = G.number_of_edges()
print(actual_edges)

10822


In [18]:
list_nodes = []
for nodes in G.nodes():
    list_nodes.append(nodes)
random.seed(42)

# Perform random sampling to get 1000 random nodes (because dataset to big)
sampled_nodes = random.sample(list_nodes, 1000)
print(len(sampled_nodes))

1000


In [19]:
G_nx = nx.Graph()
for node in sampled_nodes:
        G_nx.add_node(node)
print(G_nx.number_of_nodes())

for node in G_nx.nodes():
    node_x_neighbors = set() 
    for neighbor in G.neighbors(node):
        node_x_neighbors.add(neighbor)
    for nd in node_x_neighbors:
        if node != nd and G_nx.has_node(nd):
                G_nx.add_edge(node, nd)


1000


In [20]:
print(nx.is_connected(G_nx))

False


In [21]:
largest_cc = max(nx.connected_components(G_nx), key=len)
G_new = G_nx.subgraph(largest_cc)
print(nx.is_connected(G_new))

True


In [22]:
print(G_new.number_of_nodes())
print(G_new.number_of_edges())

128
127


In [23]:
list_nodes = [nodes for nodes in G_new.nodes()]
print(len(list_nodes))

128


In [24]:
df_new= df.loc[list_nodes]
df_target = df_new['LeaveOrNot']
df_gender = df_new['Gender']
df_education = df_new['Education']
df_joiningYear = df_new['JoiningYear']
df_City = df_new['City']
df_PaymentTier = df_new['PaymentTier']
df_Age = df_new['Age']
df_EverBenched = df_new['EverBenched']
df_ExperienceInCurrentDomain = df_new['ExperienceInCurrentDomain']

In [33]:
import pandas as pd
from scipy.sparse import coo_matrix

# Create the COO sparse matrix
df_new = df_new.apply(pd.to_numeric, errors='coerce')
row_indices, col_indices = np.where(~np.isnan(df_new.values))
coo_matrix = coo_matrix((df_new.values.ravel(), (row_indices, col_indices)), shape=(df_new.shape[0], df_new.shape[1]))

# Print the COO sparse matrix
print(coo_matrix)

  (0, 0)	0
  (0, 1)	2012
  (0, 2)	0
  (0, 3)	3
  (0, 4)	23
  (0, 5)	1
  (0, 6)	0
  (0, 7)	1
  (0, 8)	1
  (1, 0)	0
  (1, 1)	2016
  (1, 2)	0
  (1, 3)	3
  (1, 4)	27
  (1, 5)	0
  (1, 6)	1
  (1, 7)	5
  (1, 8)	0
  (2, 0)	0
  (2, 1)	2015
  (2, 2)	1
  (2, 3)	2
  (2, 4)	38
  (2, 5)	1
  (2, 6)	0
  :	:
  (125, 2)	2
  (125, 3)	3
  (125, 4)	27
  (125, 5)	1
  (125, 6)	0
  (125, 7)	5
  (125, 8)	1
  (126, 0)	0
  (126, 1)	2015
  (126, 2)	0
  (126, 3)	3
  (126, 4)	26
  (126, 5)	0
  (126, 6)	0
  (126, 7)	4
  (126, 8)	0
  (127, 0)	0
  (127, 1)	2014
  (127, 2)	0
  (127, 3)	3
  (127, 4)	40
  (127, 5)	1
  (127, 6)	0
  (127, 7)	4
  (127, 8)	0


In [34]:
# Resetting the index of the nodes from 0 to number of nodes for SINE and AE
new_indices = range(0, G_new.number_of_nodes())
mapping = {old_index: new_index for new_index, old_index in zip(new_indices, G_new.nodes())}
x_Graph = nx.relabel_nodes(G_new, mapping)

In [35]:
# paramters 
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply SINE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_SINE(x_Graph, coo_matrix,parameter_grid,  list(df_target), list(df_gender))

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [12  1]]
[[11  0]
 [ 6  0]]
done iteration
[[ 9  0]
 [12  1]]
[[11  0]
 [ 6  0]]
PPV_f and PPV_m is equal: False
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [12  1]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [12  1]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [12  1]]
[[11  0]
 [ 6  0]]
fpF: 0.0
fpM: 0.0
done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
done iteration
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
PPV_f and PPV_m is equal: True
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
fpF: 0.0
fpM: 0.0
done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0

In [36]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr_w: {best_fpr_w}")
print(f"Best Fpr_m: {best_fpr_m}")

Best Parameters: {'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 2}
Best Fpr_w: 0.0
Best Fpr_m: 0.0


In [37]:
dataframe.head

<bound method NDFrame.head of                              Female  Male
Group Fairness             1.000000   0.0
Predictive Parity          1.000000   0.0
False Positive Error Rate  0.000000   0.0
Equalized Opportunity      0.923077   1.0
Equalized Odds PPV         0.076923   0.0
Equalized Odds FPR         0.000000   0.0>

In [38]:
#Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply AE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_AE(x_Graph, coo_matrix,parameter_grid,  list(df_target), list(df_gender))

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
done iteration
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
PPV_f and PPV_m is equal: True
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
fpF: 0.0
fpM: 0.0
done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
done iteration
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
PPV_f and PPV_m is equal: True
[1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
[[ 9  0]
 [13  0]]
[[11  0]
 [ 6  0]]
fpF: 0.0
fpM: 0.0
done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0

In [39]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr_w: {best_fpr_w}")
print(f"Best Fpr_m: {best_fpr_m}")

Best Parameters: {'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 2}
Best Fpr_w: 0.0
Best Fpr_m: 0.0


In [40]:
print(dataframe)

                           Female  Male
Group Fairness                0.0   0.0
Predictive Parity             0.0   0.0
False Positive Error Rate     0.0   0.0
Equalized Opportunity         1.0   1.0
Equalized Odds PPV            0.0   0.0
Equalized Odds FPR            0.0   0.0


In [43]:
#Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'num_walks': [25, 75],
    'workers': [2, 4]
}

#apply Node2Vec embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = perform_grid_search(G_new, parameter_grid, list(df_target), list(df_gender))


Computing transition probabilities:   0%|          | 0/128 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0]
predic
[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[[5 3]
 [8 3]]
[[11  5]
 [ 3  1]]
done iteration
[[5 3]
 [8 3]]
[[11  5]
 [ 3  1]]
PPV_f and PPV_m is equal: False
[1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0]
predic
[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[[5 3]
 [8 3]]
[[11  5]
 [ 3  1]]
[[5 3]
 [8 3]]
[[11  5]
 [ 3  1]]
[[5 3]
 [8 3]]
[[11  5]
 [ 3  1]]
fpF: 0.375
fpM: 0.3125


Computing transition probabilities:   0%|          | 0/128 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 6  0]
 [11  0]]
[[14  0]
 [ 8  0]]
done iteration
[[ 6  0]
 [11  0]]
[[14  0]
 [ 8  0]]
PPV_f and PPV_m is equal: True
[1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 6  0]
 [11  0]]
[[14  0]
 [ 8  0]]
[[ 6  0]
 [11  0]]
[[14  0]
 [ 8  0]]
[[ 6  0]
 [11  0]]
[[14  0]
 [ 8  0]]
fpF: 0.0
fpM: 0.0


Computing transition probabilities:   0%|          | 0/128 [00:00<?, ?it/s]

...

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0]
predic
[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[[5 3]
 [3 5]]
[[10 11]
 [ 0  2]]
done iteration
[[5 3]
 [3 5]]
[[10 11]
 [ 0  2]]
PPV_f and PPV_m is equal: False
[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0]
predic
[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[[5 3]
 [3 5]]
[[10 11]
 [ 0  2]]
[[5 3]
 [3 5]]
[[10 11]
 [ 0  2]]
[[5 3]
 [3 5]]
[[10 11]
 [ 0  2]]
fpF: 0.375
fpM: 0.5238095238095238


Computing transition probabilities:   0%|          | 0/128 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[11  1]
 [ 8  0]]
[[13  2]
 [ 0  4]]
done iteration
[[11  1]
 [ 8  0]]
[[13  2]
 [ 0  4]]
PPV_f and PPV_m is equal: False
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0]
predic
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[11  1]
 [ 8  0]]
[[13  2]
 [ 0  4]]
[[11  1]
 [ 8  0]]
[[13  2]
 [ 0  4]]
[[11  1]
 [ 8  0]]
[[13  2]
 [ 0  4]]
fpF: 0.08333333333333333
fpM: 0.13333333333333333


Computing transition probabilities:   0%|          | 0/128 [00:00<?, ?it/s]

done create embeding vectors
[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0]
predic
[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  1]
 [10  2]]
[[13  0]
 [ 3  1]]
done iteration
[[ 9  1]
 [10  2]]
[[13  0]
 [ 3  1]]
PPV_f and PPV_m is equal: False
[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0]
predic
[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 9  1]
 [10  2]]
[[13  0]
 [ 3  1]]
[[ 9  1]
 [10  2]]
[[13  0]
 [ 3  1]]
[[ 9  1]
 [10  2]]
[[13  0]
 [ 3  1]]
fpF: 0.1
fpM: 0.0


In [44]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr_w: {best_fpr_w}")
print(f"Best Fpr_m: {best_fpr_m}")

Best Parameters: {'dimensions': 16, 'num_walks': 25, 'walk_length': 20, 'workers': 4}
Best Fpr_w: 0.0
Best Fpr_m: 0.0


In [45]:
print(dataframe)

                           Female  Male
Group Fairness                0.0   0.0
Predictive Parity             0.0   0.0
False Positive Error Rate     0.0   0.0
Equalized Opportunity         1.0   1.0
Equalized Odds PPV            0.0   0.0
Equalized Odds FPR            0.0   0.0


In [41]:
# Beispielparameter-Raster
parameter_grid = {
    'window': [16, 32],
    'walk_length': [20, 30],
    'number_of_random_walks': [25, 75],
}
#apply DeepWalk embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_f,best_fpr_m, dataframe =  perform_grid_search_DeepWalk(G_new, parameter_grid, list(df_target), list(df_gender))

25
20
16


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 669.04it/s]


[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[9 0]
 [9 1]]
[[17  1]
 [ 2  0]]
fpF: 0.0
fpM: 0.05555555555555555
[[9 0]
 [9 1]]
[[17  1]
 [ 2  0]]
PPV_f and PPV_m is equal: False
[1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[9 0]
 [9 1]]
[[17  1]
 [ 2  0]]
[[9 0]
 [9 1]]
[[17  1]
 [ 2  0]]
[[9 0]
 [9 1]]
[[17  1]
 [ 2  0]]
25
20
32


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 539.53it/s]


[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[[ 6  3]
 [10  0]]
[[15  1]
 [ 4  0]]
fpF: 0.3333333333333333
fpM: 0.0625
[[ 6  3]
 [10  0]]
[[15  1]
 [ 4  0]]
PPV_f and PPV_m is equal: True
[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[[ 6  3]
 [10  0]]
[[15  1]
 [ 4  0]]
[[ 6  3]
 [10  0]]
[[15  1]
 [ 4  0]]
[[ 6  3]
 [10  0]]
[[15  1]
 [ 4  0]]
25
30
16


...

[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 8  0]
 [11  0]]
[[16  0]
 [ 4  0]]
fpF: 0.0
fpM: 0.0
[[ 8  0]
 [11  0]]
[[16  0]
 [ 4  0]]
PPV_f and PPV_m is equal: True
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 8  0]
 [11  0]]
[[16  0]
 [ 4  0]]
[[ 8  0]
 [11  0]]
[[16  0]
 [ 4  0]]
[[ 8  0]
 [11  0]]
[[16  0]
 [ 4  0]]
75
30
16


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:01<00:00, 71.80it/s]


[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[[7 1]
 [9 0]]
[[18  1]
 [ 3  0]]
fpF: 0.125
fpM: 0.05263157894736842
[[7 1]
 [9 0]]
[[18  1]
 [ 3  0]]
PPV_f and PPV_m is equal: True
[1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[[7 1]
 [9 0]]
[[18  1]
 [ 3  0]]
[[7 1]
 [9 0]]
[[18  1]
 [ 3  0]]
[[7 1]
 [9 0]]
[[18  1]
 [ 3  0]]
75
30
32


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:01<00:00, 72.01it/s]


[1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 5  0]
 [10  0]]
[[17  0]
 [ 7  0]]
fpF: 0.0
fpM: 0.0
[[ 5  0]
 [10  0]]
[[17  0]
 [ 7  0]]
PPV_f and PPV_m is equal: True
[1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1]
predic
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[ 5  0]
 [10  0]]
[[17  0]
 [ 7  0]]
[[ 5  0]
 [10  0]]
[[17  0]
 [ 7  0]]
[[ 5  0]
 [10  0]]
[[17  0]
 [ 7  0]]


In [42]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr: {best_fpr_f}")
print(f"Best Fpr: {best_fpr_m}")
print(dataframe)

Best Parameters: {'number_of_random_walks': 25, 'walk_length': 20, 'window': 16}
Best Fpr: 0.0
Best Fpr: 0.05555555555555555
                           Female      Male
Group Fairness                0.5  0.500000
Predictive Parity             1.0  0.000000
False Positive Error Rate     0.0  0.055556
Equalized Opportunity         0.9  1.000000
Equalized Odds PPV            0.1  0.000000
Equalized Odds FPR            0.0  0.055556
